In [1]:
from scratch_model.run_scratch_model import ScratchModelEngine

In [2]:
model = ScratchModelEngine()

In [3]:
model("How can I apply?")

'You can apply to Suny Brockport by visiting their website and filling out the application.'

In [4]:
model("What are the steps for applying for financial aid?")

'The financial aid office for financial aid.'

In [5]:
del model

# Try the new one

In [6]:
model = ScratchModelEngine(model_path="models/transformer_v3")

In [7]:
model("How can I apply?")

'Applying to Suny Brockport is a straightforward first step towards applying to our vibrant community! You can start by visiting the application instructions and deadlines. Our admissions team is always ready to assist you with any questions you may have. Dont hesitate to reach out to our admissions team for guidance on how to apply!'

In [8]:
model("What are the steps for applying for financial aid?")

'Applying for financial aid is a straightforward. First, first, submit your fafsa by applying for federal student aid fafsa by the specified deadlines. Our financial aid office is ready to assist you with any questions you may have. You can find the application process on our website to get started. Dont hesitate to reach out for guidance on applying!'